In [1]:
!pip install transformers[torch]
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 4.9 MB/s eta 0:00:00


In [2]:
#import warnings
#warnings.filterwarnings("ignore", category=UserWarning, module='transformers.generation.utils')

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
path='/content/drive/MyDrive/GYAFC_Corpus'

In [5]:
import pandas as pd

In [6]:
df_test=pd.DataFrame()

In [7]:
df_test['informal']=pd.read_csv('drive/MyDrive/GYAFC_Corpus/Entertainment_Music/test/informal',sep='\t')

In [8]:
df_test['formal']=pd.read_csv('drive/MyDrive/GYAFC_Corpus/Entertainment_Music/test/formal',sep='\t')

In [9]:
df_train_Entertainment_Music=pd.DataFrame()

In [10]:
df_train_Family_Relationships=pd.DataFrame()

In [11]:
df_train_Entertainment_Music['informal']=pd.read_csv('drive/MyDrive/GYAFC_Corpus/Entertainment_Music/train/informal',sep='\t')

In [12]:
df_train_Entertainment_Music['formal']=pd.read_csv('drive/MyDrive/GYAFC_Corpus/Entertainment_Music/train/formal',sep='\t')

In [13]:
df_train_Family_Relationships['informal']=pd.read_csv('drive/MyDrive/GYAFC_Corpus/Family_Relationships/train/informal',sep='\t')

In [14]:
df_train_Family_Relationships['formal']=pd.read_csv('drive/MyDrive/GYAFC_Corpus/Family_Relationships/train/formal',sep='\t')

In [15]:
df_train= pd.concat([df_train_Entertainment_Music,df_train_Family_Relationships], ignore_index=True)

In [16]:
df_train.head(-1)

,informal,formal
0,that page did not give me viroses(i think),I don't think that page gave me viruses.
1,"of corse i be wachin it evry day, my fav chara...","I watch it everyday, my favorite charachter is..."
2,runescape.com (my kids love it) & funbrain.com...,Funbrain.com and runescape.com are great for f...
3,Is he gay?He was on Late Night with Conan O'Br...,He was on the Late Night show with Conan O'Bri...
4,"id have to say mel gibson, a strong [god] beli...",Mel Gibson is a strong believed of God.
...,...,...
104554,is he alot older than you?,Is he much older than you are?
104555,of corse it depends on what relation u r looki...,"Of course, it depends on what type of relation..."
104556,Wear a sign that says Hi!,"Wear a sign that say ""Hi!"""
104557,"I don't believe in playing games, I hate when ...",I do not like when guys play games with me.


In [17]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104560 entries, 0 to 104559
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   informal  104560 non-null  object
 1   formal    104558 non-null  object
dtypes: object(2)
memory usage: 1.6+ MB


In [18]:
df_train = df_train.dropna(subset=['informal', 'formal'])

In [19]:
df_test.head()

,informal,formal
0,"aspen colorado has he best music festivals, yo...",There's nothing he needs to change.
1,You can get almost anything on ebay!,It does not exist.
2,everybody is Dying to get in,Mine is book by Steve Martin called 'The Pleas...
3,not idiots like 50 cent and his whole Gay unit...,What differentiates a mosquitoo from a blonde?
4,different from what I've seen though,"They're pretty good. Also, that's a good song."


In [20]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1415 entries, 0 to 1414
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   informal  1415 non-null   object
 1   formal    1081 non-null   object
dtypes: object(2)
memory usage: 22.2+ KB


In [21]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments

In [22]:
df_train.head()

,informal,formal
0,that page did not give me viroses(i think),I don't think that page gave me viruses.
1,"of corse i be wachin it evry day, my fav chara...","I watch it everyday, my favorite charachter is..."
2,runescape.com (my kids love it) & funbrain.com...,Funbrain.com and runescape.com are great for f...
3,Is he gay?He was on Late Night with Conan O'Br...,He was on the Late Night show with Conan O'Bri...
4,"id have to say mel gibson, a strong [god] beli...",Mel Gibson is a strong believed of God.


In [23]:
df_train['combined'] = df_train['informal'] + " [SEP] " + df_train['formal']
df_train['combined'].to_csv('training_data.txt', index=False, header=False)

In [24]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model_gpt2 = GPT2LMHeadModel.from_pretrained('gpt2')

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [25]:
train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path='training_data.txt',
    block_size=128
)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [26]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

In [27]:
#remember to restart the session before running below cells

In [28]:
training_args = TrainingArguments(
    output_dir='./gpt2-formal',
    overwrite_output_dir=True,
    num_train_epochs=10,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
)

In [29]:
trainer = Trainer(
    model=model_gpt2,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

In [30]:
trainer.train()

Step,Training Loss
500,3.015400
1000,2.857700
1500,2.831000
2000,2.770300
2500,2.769500
3000,2.739100
3500,2.730400
4000,2.724800
4500,2.705000
5000,2.704300


Step,Training Loss
500,3.015400
1000,2.857700
1500,2.831000
2000,2.770300
2500,2.769500
3000,2.739100
3500,2.730400
4000,2.724800
4500,2.705000
5000,2.704300


TrainOutput(global_step=65900, training_loss=2.2612517208394585, metrics={'train_runtime': 4511.3916, 'train_samples_per_second': 58.428, 'train_steps_per_second': 14.607, 'total_flos': 1.721849167872e+16, 'train_loss': 2.2612517208394585, 'epoch': 10.0})

In [36]:
model_gpt2.save_pretrained('./gpt2-formal')
tokenizer.save_pretrained('./gpt2-formal')

('./gpt2-formal/tokenizer_config.json',
 './gpt2-formal/special_tokens_map.json',
 './gpt2-formal/vocab.json',
 './gpt2-formal/merges.txt',
 './gpt2-formal/added_tokens.json')

In [37]:
tokenizer = GPT2Tokenizer.from_pretrained('./gpt2-formal')
model_gpt2 = GPT2LMHeadModel.from_pretrained('./gpt2-formal')

In [38]:
df_train.head()

,informal,formal,combined
0,that page did not give me viroses(i think),I don't think that page gave me viruses.,that page did not give me viroses(i think) [SE...
1,"of corse i be wachin it evry day, my fav chara...","I watch it everyday, my favorite charachter is...","of corse i be wachin it evry day, my fav chara..."
2,runescape.com (my kids love it) & funbrain.com...,Funbrain.com and runescape.com are great for f...,runescape.com (my kids love it) & funbrain.com...
3,Is he gay?He was on Late Night with Conan O'Br...,He was on the Late Night show with Conan O'Bri...,Is he gay?He was on Late Night with Conan O'Br...
4,"id have to say mel gibson, a strong [god] beli...",Mel Gibson is a strong believed of God.,"id have to say mel gibson, a strong [god] beli..."


In [ ]:
input_sentence = "yo what r u doin"
input_ids = tokenizer.encode(input_sentence, return_tensors='pt')
output = model_gpt2.generate(input_ids)
print(tokenizer.decode(output[0], skip_special_tokens=True))

In [43]:
input_sentence = "that page did not give me viroses(i think)"
input_ids = tokenizer.encode(input_sentence, return_tensors='pt')

# Create attention mask
attention_mask = torch.ones(input_ids.shape, dtype=torch.long) # Create a mask of 1s for each token followed by 0s for padding

# Generate output with attention mask
output = model_gpt2.generate(input_ids, attention_mask=attention_mask)

print(tokenizer.decode(output[0], skip_special_tokens=True))


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


that page did not give me viroses(i think) [SEP] I believe the
